In [1]:
import os

In [2]:
pwd = os.getcwd()
print(pwd)

/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/research


In [3]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/research'

In [4]:
os.chdir('../')
# os.chdir('/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2')


In [5]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2'

In [6]:
import pandas as pd

In [7]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [11]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # dataclass automatically generates __init__ and __repr__ etc special methods, based on class variables. # 'frozen=True' makes the class immutable---> ((once the object of the class is created you cannot modify its attributes.)) and here these attributes are the class variables.
class DataValidationConfig:
    root_dir: Path #a variable that stores the root directory (of type Path, which is likely from the pathlib library) where the data or configurations are stored.
    unzip_data_dir: Path # stores the directory path (also a Path object) where data that has been unzipped is located or will be stored.
    STATUS_FILE: str # A string that holds the filename or path of a file where the status of the validation process might be stored
    all_schema: dict # A dictionary that holds schema information, which likely defines the expected structure of the data (such as column names, data types, constraints, etc.).

In [12]:
from MlOpsProject.constants import *
from MlOpsProject.utils.common import read_yaml, create_directories

In [13]:
# Update the Configuration Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.Columns
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema
        )
        
        return data_validation_config

In [14]:
# update the Components 

import os
from MlOpsProject import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e
        

In [15]:
import os
print(os.getcwd())

/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2


In [16]:
import os
print(os.environ)

environ({'ALACRITTY_LOG': '/var/folders/h9/npfdffz11hbbc6s4ymyf9j0h0000gn/T/Alacritty-28813.log', 'ALACRITTY_SOCKET': '/var/folders/h9/npfdffz11hbbc6s4ymyf9j0h0000gn/T/Alacritty-28813.sock', 'ALACRITTY_WINDOW_ID': '4917912992', 'AUTOJUMP_ERROR_PATH': '/Users/sanket/Library/autojump/errors.log', 'AUTOJUMP_SOURCED': '1', 'COLORTERM': 'truecolor', 'COMMAND_MODE': 'unix2003', 'CONDA_DEFAULT_ENV': 'myMlProj', 'CONDA_EXE': '/opt/anaconda3/bin/conda', 'CONDA_PREFIX': '/opt/anaconda3/envs/myMlProj', 'CONDA_PROMPT_MODIFIER': '(myMlProj) ', 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python', 'CONDA_SHLVL': '2', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'GSETTINGS_SCHEMA_DIR': '/opt/anaconda3/share/glib-2.0/schemas', 'HADOOP_COMMON_HOME': '/Users/sanket/hadoop-3.4.0', 'HADOOP_COMMON_LIB_NATIVE_DIR': '/Users/sanket/hadoop-3.4.0/lib/native', 'HADOOP_HDFS_HOME': '/Users/sanket/hadoop-3.4.0', 'HADOOP_HOME': '/Users/sanket/hadoop-3.4.0', 'HADOOP_INSTALL': '/Users/sanket/hadoop-3.4.0', 'HADOOP_MAPRED_HOME': '/Us

In [17]:
import os
print(os.listdir('config'))


['config.yaml']


In [18]:
import os
print(os.getcwd())  # Check the current working directory

/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2


In [19]:
from box import ConfigBox

In [20]:
import sys
print(sys.path)

['/opt/anaconda3/envs/myMlProj/lib/python312.zip', '/opt/anaconda3/envs/myMlProj/lib/python3.12', '/opt/anaconda3/envs/myMlProj/lib/python3.12/lib-dynload', '', '/opt/anaconda3/envs/myMlProj/lib/python3.12/site-packages', '/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/src', '/opt/anaconda3/envs/myMlProj/lib/python3.12/site-packages/setuptools/_vendor']


In [21]:
import sys
sys.path.append('/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/src')

In [22]:
# Execution logic
try:
    config = ConfigurationManager()  # Initialize configuration manager
    data_validation_config = config.get_data_validation_config()  # Get data validation config
    data_validation = DataValidation(config=data_validation_config)  # Corrected typo here
    data_validation.validate_all_columns()  # Validate all columns
except Exception as e:
    raise e  # Raise any exceptions encountered

[2024-10-04 23:48:36,619: INFO: common: YAML file: config/config.yaml loaded successfully.]
[2024-10-04 23:48:36,624: INFO: common: YAML file: params.yaml loaded successfully.]
[2024-10-04 23:48:36,627: INFO: common: YAML file: schema.yaml loaded successfully.]
[2024-10-04 23:48:36,628: INFO: common: created directory at: artifacts]
[2024-10-04 23:48:36,629: INFO: common: created directory at: artifacts/data_validation]
